<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/PyTorch_Projects/blob/main/custom_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recap

Steps to solve a machine learning problem:
1. Find a dataset
2. Turn the dataset into numbers
3. Create a model or find a pretrained one to extract the patterns in the data for prediction

# Intro

- Custom Data: a collection of data relating to your problem.
- PyTorch provides functions to load in different custom datasets in the **TorchVision**, **TorchText**, **TorchAudio**, and **TorchRec** domain libraries.
- If the previous functions are not enough, we can subclass **torch.utils.data.Dataset** and customize it to our needs.
- In this notebook, we will not use an in-built PyTorch dataset.
- We will load a custom dataset and train a model on it.



# Setting up environment

In [1]:
import torch
from torch import nn

torch.__version__

'2.6.0+cu124'

In [2]:
# Using a device-agnostic code is a best practice in DL

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# Setting up Data

- Start with existing data.
- We will use a subset of the [Food-101](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/) dataset.
- Food-101 splits to:
    - 1000 images of 101 different kinds of foods = 101,000 total images
    - 750,750 for training and 250,250 for testing
- Food-101 is included in PyTorch now.

## Formatting the custom data

- We will start with three food classes: Sushi, Pizza, and Steak.

### Data Creation

In [3]:
import torchvision as tv
import pathlib # Setting up data directory

tv.__version__
data_dir = pathlib.Path("../data")

In [ ]:
train_data = tv.datasets.Food101(root = data_dir,
                         split = "train",
                         download = True)

test_data = tv.datasets.Food101(root = data_dir,
                                split = "test",
                                download = True)

100%|██████████| 5.00G/5.00G [02:47<00:00, 29.8MB/s]


In [ ]:
train_data, test_data

In [ ]:
class_names = train_data.classes
print(len(class_names))
class_names[:20]

In [ ]:
# View a sample image

print(class_names[train_data[1000][1]])
train_data[1000][0]

### Subseting appropriate classes

- Copy 10% random images of the specified classes to a separate folders.

In [ ]:
import random

In [ ]:
data_path = data_dir / "food-101" / "images"
target_classes = ["sushi", "steak", "pizza"]

DATA_AMOUNT = 0.1

def get_subset(image_path = data_path,
               data_splits = ['train', 'test'],
               target_classes = ['pizza', 'steak', 'sushi'],
               amount = 0.1,
               seed = 42):

    random.seed(seed)
    label_splits = {}

    for data_split in data_splits:

        print(f"[INFO] Creating {data_split} split..")

        label_path = data_dir / "food-101" / "meta" / f"{data_split}.txt"
        with open(label_path, 'r') as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in target_classes]

        # Extracting a random subset
        n_samples = round(amount * len(labels))
        print(f"[INFO] Extracting random subset of {n_samples} images for {data_split}..")
        sample_images = random.sample(labels, k = n_samples)

        # Apply full path
        image_paths = [pathlib.Path(str(image_path / sample_image) + ".jpg") for sample_image in sample_images]
        label_splits[data_split] = image_paths

    return label_splits

In [ ]:
label_splits = get_subset(amount = DATA_AMOUNT)
label_splits.keys(), label_splits['test'][:14]

### Moving images to dedicated folders

In [ ]:
# Create target directory path
target_dir_name = f"../data/pizz_sushi_steak_{str(int(DATA_AMOUNT*100))}_precent"
print(f"Creating directory: '{target_dir_name}'")

# Set up the directories
target_dir = pathlib.Path(target_dir_name)
# Make the directories
target_dir.mkdir(parents = True, exist_ok = True)

In [ ]:
import shutil

In [ ]:
print(target_dir)

In [ ]:
for data_split in label_splits.keys():
    for image_path in label_splits[str(data_split)]:
        print(type(image_path))
        print(image_path.parent)
        print(image_path.parent.stem)
        print(image_path.name)
        dest_dir = target_dir / data_split / image_path.parent.stem / image_path.name
